In [2]:
#!value --name example01
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11

In [3]:
#!share --from value example01

open System.Text.RegularExpressions

let splitter = Regex(@"\s+").Split

type Card = {
    no: int
    winning: float seq
    numbers: float seq
}
module Card =
    let wins (card:Card) =
        card.winning |> Set.ofSeq
        |> Set.intersect (card.numbers |> Set.ofSeq)
        |> Set.count

    let points (card:Card) =
        card
        |> wins
        |> (fun it -> 
            let result = 2.0 ** (float it - 1.0)
            if result >= 1.0 then result else 0.0   
        )
        
let parseNums (nums:string) =
    nums
    |> splitter
    |> Seq.filter(String.IsNullOrEmpty >> not )
    |> Seq.map (fun it -> it.Trim())
    |> Seq.map float

let parseLine (line:string) =
    let parts = line.Split(": ")
    let game = (splitter parts.[0]).[1].Trim() |> int
    let nums = parts.[1].Split(" | ")
    let winning = nums.[0] |> parseNums
    let numbers = nums.[1] |> parseNums
    { no = game; winning = winning; numbers = numbers }

let part1 = Seq.map parseLine >> Seq.map Card.points >> Seq.sum

example01.Split("\n")
|> part1

13

In [4]:
File.ReadAllText(__SOURCE_DIRECTORY__ + "/input01.txt").Split("\n")
|> part1

26218

In [21]:
let expand (cards:int list):int list =
        let rec recurse cards =
            match cards with 
            | [] -> []
            | head::tail ->
                if head = 0 then
                    [head]
                else
                seq {
                    for x in [0 .. (head-1)] do
                        let copy = tail |> List.skip x |> recurse 
                        yield copy

                } 
                |> Seq.concat
                |> Seq.append [head]
                |> List.ofSeq
        recurse cards


let part2 =
    Seq.map parseLine
    >> Seq.map Card.wins
    >> List.ofSeq
    >> expand
    >> Seq.length

// 4 2 2 1 0 0
example01.Split("\n")
|> part2

77

In [136]:
File.ReadAllText(__SOURCE_DIRECTORY__ + "/input01.txt").Split("\n")
|> part2

Error: Command cancelled.